In [0]:
from datetime import datetime, timedelta
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType

In [0]:

# Формування дати для зчитування файлу
date_previous = (datetime.today()- timedelta(days=1)).strftime('%Y_%m_%d')
file_path = f"/FileStore/tables/stock_prices_{date_previous}.csv"  # /dbfs для Python-level доступу

# Визначаємо: чи це історичний (заголовок) чи щоденний файл (без заголовка)
first_file = "2025_06_10" in file_path  

# Читаємо з відповідним header-параметром
bronze_df = (
    spark.read
    .option("header", first_file)  # True — якщо файл з заголовком, False — якщо без
    .option("inferSchema", False)  # Схема не визначається автоматично
    .option("mergeSchema", "true")
    .csv(file_path)
)

# Якщо файл без заголовка, явно задаємо назви колонок
if not first_file:
    column_names = ["Date", "Company", "Open", "High", "Low", "Close", "Volume"]
    bronze_df = bronze_df.toDF(*column_names)



In [0]:
# Запис сирих(raw) даних у delta таблицю "bronze_stock"
bronze_df.write.format("delta").mode("append").option("mergeSchema", "true").save("/FileStore/delta/bronze_stock")

In [0]:

#Визначення схеми для таблиці логування

log_schema = StructType([
    StructField("timestamp", StringType(), False),
    StructField("pipeline_level", StringType(), False),
    StructField("source_file", StringType(), True),
    StructField("record_count", IntegerType(), True),
    StructField("null_count", IntegerType(), True),
    StructField("duplicates_removed", IntegerType(), True),
    StructField("gold_daily_count", IntegerType(), True),
    StructField("gold_weekly_count", IntegerType(), True),
    StructField("gold_monthly_count", IntegerType(), True)
])


In [0]:
#Додавання даних до лог-таблиці
log_data = [(
    datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    "bronze_stock_data",
    file_path,
    bronze_df.count(),
    None,
    None,
    None,
    None,
    None
)]

log_bronze_df = spark.createDataFrame(log_data, schema=log_schema)
#Запис даних до лог-таблиці "etl_log" у форматі delta
log_bronze_df.write.format("delta").mode("append").save("/FileStore/delta/etl_log")